### Robust optimization with Pyomo and Romodel

#### ROmodel installation

Use following command in your terminal (eg. Mac terminal, Anaconda prompt)

pip install git+https://github.com/cog-imperial/romodel.git

#### Example problem

$$ \max_{}x_1 + x_2 $$
$$ s.t. \; (5+u_1)x_1 + (10+2u_2)x_2 \leq 100 $$
$$ x_1,x_2 \geq 0 $$
$$ u_1,u_2\in U $$
$$ U\equiv \left\{ u_1,u_2| \left| u_1\right| \leq 1, \left| u_2\right| \leq 1  \right\} $$

In [50]:
import pyomo.environ as pe
import romodel as ro
from romodel import uncset

In [57]:
m = pe.ConcreteModel()

m.x1 = pe.Var(within=pe.NonNegativeReals)
m.x2 = pe.Var(within=pe.NonNegativeReals)

m.obj = pe.Objective(expr=m.x1 + m.x2, sense=pe.maximize)

In [58]:
m.uc_set = uncset.PolyhedralSet(mat=[[ 1,  0],
                                    [ 0,  1],
                                    [-1,  0],
                                    [ 0, -1]],
                                rhs=[1, 1, 1, 1])
m.uc_set_1 = uncset.PolyhedralSet(mat=[[ 1],
                                       [-1]],
                                  rhs=[1, 1,])
m.u1 = ro.UncParam( uncset=m.uc_set_1, nominal=[0.0,0.0])

m.uc_set_2 = uncset.PolyhedralSet(mat=[[ 1],
                                       [-1]],
                                  rhs=[1, 1,])
m.u2 = ro.UncParam( uncset=m.uc_set_2, nominal=[0.0,0.0])

In [59]:
def con_rule(m):
    return (5+m.u1)*m.x1 + (10+2*m.u2)*m.x2 <= 100
m.con = pe.Constraint( rule=con_rule )

In [60]:
solver = pe.SolverFactory('romodel.cuts')
solver.options.solver = "cbc"
solver.solve(m)

Adding 1 cutting plane generators.
Using solver cbc

Solving nominal problem.

0/1 constraints robustly feasible. Add cuts and resolve.
1/1 constraints robustly feasible. Add cuts and resolve.

All constraints robustly feasible after 2 iterations.


{'Problem': [{'Name': 'unknown', 'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': None, 'Number of constraints': None, 'Number of variables': None, 'Number of binary variables': None, 'Number of integer variables': None, 'Number of continuous variables': None, 'Sense': 'unknown'}], 'Solver': [{'Name': None, 'Status': 'ok', 'Wallclock time': 1.3671867847442627, 'Termination condition': 'optimal'}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [62]:
print("Objective : ", m.obj())
print("x1: ", m.x1(), " x2: ", m.x2())

Objective :  16.666667
x1:  16.666667  x2:  0.0
